In [2]:
import os

# print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



In [3]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma


embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
persist_dir = './database'
vector_store = Chroma(
    persist_directory=persist_dir,  # 있으면 가져오고 없으면 생성
    embedding_function=embeddings
)

In [10]:
import pickle

with open('all_docs.pkl', 'rb') as f:
    all_docs = pickle.load(f)

In [12]:
len(vector_store.get()['ids'])

1695

In [13]:
bm25_retriever = BM25Retriever.from_documents(
    all_docs,
)
bm25_retriever.k = 5

In [14]:
chroma_retriever = vector_store.as_retriever(
    search_kwargs = {"k": 5},
    search_type   = "mmr"
)

In [15]:
# Ensenble retriever 선언
ensemble_retriever = EnsembleRetriever(

    retrievers=[bm25_retriever, chroma_retriever],
    weights=[0.65, 0.35], # 가중치
    search_type='mmr', # mmr 기반
)

### chroma retriever, ensemble retriever 비교

In [16]:
query = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."

In [17]:
chroma_retriever.get_relevant_documents(query)

C:\Users\User\anaconda3\envs\BP\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='502023 재외공관의  해외진출기업 지원 사례집\n■특히, 사실이 아닌 루머로 인해 불안감을 키우는 경우가 있는 만\n큼, 대사관으로서는 주재국 노동/산업 당국과 긴밀한 관계를 구축하고, 우리 동포 기업인을 대상으로 대사관이 파악할 수 있는 최신 규제 관련 정보나 동향을 정기적으로 공유할 필요가 있음. ', metadata={'page': 49, 'source': 'data/PDF_with_contents/3. 정책 및 무역\\[국가지원사례][외교부][2023]재외공관의 해외진출기업 지원 사례집.pdf'}),
 Document(page_content='사례로 정리해보는\n한-인도 CEPA \n활용법 및 인도 통상 애로\nKOTRA자료 22-055', metadata={'page': 0, 'source': 'data/PDF_with_contents/3. 정책 및 무역\\[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf'}),
 Document(page_content='145\n제Ⅳ절 | 인도 통상정책1. 반덤핑·상계관세·긴급수입제한조치 조사 사례 조회방법\n홈페이지 접속 http://www.dgtr.gov.in/\n2. 확인하고자 하는 무역구제(반덤핑·상계관세·긴급수입제한조치) 조사 선택\n① 반덤핑 조사(Antidumping Investigations)\n② 상계관세 조사(Countervailing Duty Investigation)\n③ 긴급수입제한조치 조사(Safeguard Investigation in India)\n\ue194\ue192\n\ue193', metadata={'page': 146, 'source': 'data/PDF_with_contents/3. 정책 및 무역\\[정책][관세청][2019.11.07]신남방국 통관·통상환경 및 FTA 활용방안.pdf'}),
 Document(page_content='259Ⅰ\nⅡ\nⅢ\nⅣ\nⅤ\nⅥ인도대사관\n인도대사관\n

In [18]:
ensemble_retriever.get_relevant_documents(query)

[Document(page_content='인도 수출 및 CEPA\n활용 시 주의사항 알아두기   01\n1. 품목분류  \n가. HS코드 관련 인도세관의 품목분류 차이 발생 시 원산지증명서 발급 방법\n나. 인도 DRI(국세정보국) 관세조사 시 대응방안\n2. 원산지증명서  \n가. 원산지증명서 오류 발급 시 정정 방법\n나. 원산지증명서 원본 여부 논란 발생 시 대응방안\n3. CEPA 활용  \n가. 인도 CAROT AR 2020(원산지관리강화규칙)에 의한 Form I 작성 및 제출 방법\n나. 2022년 HS협약 개정에 따라 HS코드가 변경된 물품의 수출 시 주의사항\n다. 중고 설비 인도 수출 시 한-인도 CEPA 활용 방법사례로 정리해보는 한-인도\nCEPA 활용법 및 인도 통상 애로\n', metadata={'source': 'data/PDF_with_contents/3. 정책 및 무역\\[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'page': 3}),
 Document(page_content='08 | 사례로 정리해보는 한-인도 CEPA 활용법 및 인도 통상 애로\n기재하여 신청하면 가능하다. 다만, 한국과 인도의 양국에서 사용하는 HS 코드에 따른 원산지기준이 다를 경\n우, 원산지기준을 모두 충족하여야 한다.\n나. 인도 DRI(국세정보국) 관세조사 시 대응방안\n① 주요 내용\n2021년 3월, 철도 신호용 계전기 관련 인도 대법원의 판결(Westinghouse Saxby FarmerLtd. Vs. \nCommisioner of Central Excise)에 따라 개별 부품이 자동차의 부품으로 분류될 근거가 됨으로써 인도 국\n세정보국(DRI)는 자동차 부품 관련 자의적 해석에 근거한 관세조사를 시행하고 있다. 이로 인해, 인도에 진\n출한 관련 기업들의 한-인도 CEPA  활용 불가 및 차액 관세 납부 등의 피해가 발생하고 있다. 한-인도 CEPA 적용을 위한 원산지증명서 서식\n자료: 한

## TESTING

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder



# 사용자 질문 맥락화 프롬프트
contextualize_q_system_prompt = """
주요 목표는 사용자의 질문을 이해하기 쉽게 다시 작성하는 것입니다.
사용자의 질문과 채팅 기록이 주어졌을 때, 채팅 기록의 맥락을 참조할 수 있습니다.
채팅 기록이 없더라도 이해할 수 있는 독립적인 질문으로 작성하세요.
질문에 바로 대답하지 말고, 필요하다면 질문을 다시 작성하세요. 그렇지 않다면 질문을 그대로 반환합니다.        
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# 질문 프롬프트
qa_system_prompt = """
당신은 질문에 대해 정보를 제공해주는 어시스턴트입니다.
소규모 기업 혹은 스타트업에 인도에 수출 사업을 도와 사실 기반의 정보만을 제공해주면서 가능한 한 도움이 되도록 만들어졌습니다.
사전 정보가 아닌 주어진 자료를 참고하여 정보를 제공해주세요.
만약 자료에 나와있지 않는 상황의 경우에는 인터넷에 검색해보라고 말해주세요.

자료:
{context}

질문:
{input}

FORMAT:
- 진출 사업
- 진출 사업 동향
- 진출 사업 메리트
- 진출 사업 관련 정책
- 진출 사업을 위한 도움말
- 진출 사업 관련 알아야 할 것들
"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

In [20]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0,
)

In [21]:
# from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from operator import itemgetter
output_parser = StrOutputParser()


history_aware_retriever = create_history_aware_retriever(
    llm = llm,
    retriever = ensemble_retriever,
    prompt = contextualize_q_prompt
)

# 응답 생성 + 프롬프트 엔지니어링
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



In [22]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x0000015C1A9D3F10>, k=5), VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000015C76AFF2E0>, search_type='mmr', search_kwargs={'k': 5})], weights=[0.65, 0.35]))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(inpu

In [23]:
query = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."
result = rag_chain.invoke(
    {
        "input": query,
        "chat_history": [],
    }
)

In [24]:
result['answer']

'- 진출 사업: 한-인도 CEPA를 활용한 수출 사업\n- 진출 사업 동향: 인도 DRI(국세정보국)의 관세조사가 자동차 부품 관련 부분적 해석에 근거하여 실시되고 있음.\n- 진출 사업 메리트: 한-인도 CEPA를 활용하여 관세 혜택을 받을 수 있으며, 수출 시 원활한 절차 진행이 가능함.\n- 진출 사업 관련 정책: 2022년 1월에 인도 관세간접세위원회(CBIC)가 자의적 품목분류를 규제하기 위한 지침을 발표함.\n- 진출 사업을 위한 도움말: 관세 관련 고시 및 지침은 CBIC의 공식 웹사이트에서 확인 가능하며, 관련 기업은 근거자료를 확보하여 대응할 필요가 있음.\n- 진출 사업 관련 알아야 할 것들: 부품(Parts)의 품목분류 시 HS 협약에 따른 규정을 우선 적용하고 다양한 사례를 참고하여 판단해야 함. 인도 DRI의 관세조사에 대비하여 적절한 대응 방안을 마련해야 함.'

In [25]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
result = rag_chain.invoke(
    {
        "input": query,
        "chat_history": [],
    }
)

In [26]:
result['answer']

'원산지증명서 오류 발급 시 정정 방법은 다음과 같습니다:\n\n1. 원산지증명서 오류 발생 시, 먼저 해당 오류를 파악하고 정정이 필요한 부분을 확인합니다.\n2. 정정이 필요한 부분을 수정하여 정정된 원산지증명서를 발급해야 합니다.\n3. 정정된 원산지증명서를 발급하기 위해서는 기존에 발급된 원산지증명서의 원본을 회수해야 합니다.\n4. 한-인도 CEPA 협정을 적용하기 위해서는 정정된 원산지증명서를 제출해야 합니다.\n5. 정정된 원산지증명서를 제출한 후, 인도 세관의 심사를 거쳐 CEPA 혜택을 받을 수 있습니다.\n\n이러한 절차를 통해 원산지증명서 오류를 정정하고, 한-인도 CEPA 혜택을 적용할 수 있습니다.'

In [48]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}  # 세션 기록을 저장할 딕셔너리


def get_session_history(session_ids: str) -> BaseChatMessageHistory:
    print(session_ids)
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


with_message_history = (
    RunnableWithMessageHistory(  # RunnableWithMessageHistory 객체 생성
        rag_chain,  # 실행할 Runnable 객체
        get_session_history,  # 세션 기록을 가져오는 함수
        input_messages_key="input",  # 입력 메시지의 키
        history_messages_key="history",  # 기록 메시지의 키
    )
)

In [49]:
query = "원산지증명서 오류 발급 시 정정 방법을 알려줘"
SESSION_ID = "abc123"

result = with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {
        "input": query,
        "chat_history": [],
    },
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": SESSION_ID}},
)

abc123


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


In [50]:
result['answer']

'원산지증명서 오류 발급 시 정정 방법은 다음과 같습니다:\n\n1. 한국 세관에 원산지증명서 정정신청을 합니다.\n2. 정정된 원산지증명서를 제출하고, 한-인도 CEPA를 적용합니다.\n3. 원산지증명서의 원본(오류 발급본)을 회수합니다.\n\n이러한 절차를 통해 오류가 있는 원산지증명서를 정정하고, 한-인도 CEPA를 적용할 수 있습니다.원산지증명서의 정정은 신속하게 처리되어야 하며, 관련된 모든 당사자들에게 적절히 통보되어야 합니다.'

In [51]:
query = "인도 DRI(국세정보국) 관세조사 시 대응방안에 대한 사례를 알려주세요."
SESSION_ID = "abc123"

result = with_message_history.invoke(
    # 수학 관련 질문 "코사인의 의미는 무엇인가요?"를 입력으로 전달합니다.
    {
        "input": query,
        "chat_history": [],
    },
    # 설정 정보로 세션 ID "abc123"을 전달합니다.
    config={"configurable": {"session_id": SESSION_ID}},
)

result['answer']

abc123


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


'**진출 사업:** 인도 DRI(국세정보국) 관세조사 시 대응방안\n\n**진출 사업 동향:** 인도 DRI(국세정보국)의 관세조사는 자동차 부품 관련 자의적 해석에 근거한 조사를 시행하고 있으며, 이로 인해 한-인도 CEPA 활용 불가 및 차액 관세 납부 등의 피해가 발생하고 있습니다.\n\n**진출 사업 메리트:** 대비책을 마련하여 인도 DRI(국세정보국)의 관세조사에 대응할 수 있다면, 한-인도 CEPA 혜택을 최대한 활용하고 추가 관세 부담을 최소화할 수 있습니다.\n\n**진출 사업 관련 정책:** 2022년 1월 5일, 인도 관세간접세위원회(CBIC)가 세관 및 국세정보국(DRI)의 자의적 품목분류를 규제하기 위한 지침을 발표하였습니다.\n\n**진출 사업을 위한 도움말:** 관련 기업들은 근거자료를 확보하고, HS 협약에 따른 규정을 우선 적용하며, 다양한 사례를 종합적으로 참고하여 판단하는 것이 중요합니다.\n\n**진출 사업 관련 알아야 할 것들:** 인도 DRI(국세정보국)의 관세조사에 대비하기 위해서는 정확한 원산지증명서를 제출하고, HS 코드에 대한 이해와 관련 서류를 준비하는 것이 중요합니다. 또한, 인도 관세간접세위원회(CBIC)의 공식 웹사이트를 통해 관세 관련 고시 및 지침을 확인하는 것이 도움이 될 수 있습니다.'

In [61]:
embeddings = OpenAIEmbeddings( model="text-embedding-ada-002" )    
# 나중에 허깅페이스에서 임베딩 모델 가져오기 

persist_dir = "./database"

vector_store = Chroma(
    persist_directory=persist_dir,  # 있으면 가져오고 없으면 생성
    embedding_function=embeddings
)



C:\Users\User\anaconda3\envs\BP\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
vector_store.add_documents()

In [65]:
docs[0]

Document(page_content='사례로 정리해보는\n한-인도 CEPA \n활용법 및 인도 통상 애로\nKOTRA자료 22-055\n', metadata={'source': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'file_path': './data/[법률_규범_특허][코트라][2022]한-인도 CEPA 활용법 및 인도 통상 애로.pdf', 'page': 0, 'total_pages': 41, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 17.2 (Macintosh)', 'producer': 'Adobe PDF Library 16.0.7', 'creationDate': "D:20220610145706+09'00'", 'modDate': "D:20220614154828+09'00'", 'trapped': ''})

In [66]:
docs[0].page_content

'사례로 정리해보는\n한-인도 CEPA \n활용법 및 인도 통상 애로\nKOTRA자료 22-055\n'

In [69]:
docs[0].get()

AttributeError: 'Document' object has no attribute 'get'